<a href="https://colab.research.google.com/github/UVARCteam/2023_5th_UVARC_workshop/blob/main/test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

w1 = torch.zeros(1,requires_grad=True)
w2 = torch.zeros(1,requires_grad=True)
w3 = torch.zeros(1,requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SDG([w1,w2.w3.b], lr = 1e-05)

nb_epoch = 1000

for epoch in range(nb_epoch+1):




In [6]:
w1

tensor([0.], requires_grad=True)